In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
from pathlib import Path
import seaborn as sns
from PIL import Image
import numpy as np
import cv2 as cv
from skimage import io, exposure, img_as_float
from matplotlib import animation, rc
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
rc('animation', html='jshtml')

In [ ]:
df = pd.read_csv('train_rles.csv')
df['subset']=df['id'].map(lambda x:'_'.join(x.split('_')[:-1]))
df

In [ ]:
print("Number of unique slices:", df['id'].nunique())
print("Number of unique datasets:", df['subset'].nunique())

In [ ]:
df['mask_length'] = df['rle'].apply(lambda x: len(str(x).split()))
df['scan'] = df['id'].apply(lambda x: x[:-5])
df['mask_is_empty'] = df['id'].apply(lambda x: x[:-5])

df['scan'] = df['id'].apply(lambda x: x[:-5])
df['mask_is_empty'] = df['rle']=='1 0'
df

In [ ]:
df['subset'].value_counts().plot.bar()

In [ ]:
def load_image(file_path):
    with Image.open(file_path) as img:
        return np.array(img)

def load_v2(path):
    im = Image.open(path)
    im = im.convert('RGBA')
    
    data = np.array(im)  
    red, green, blue, alpha = data.T 

    white_areas = (red == 255) & (blue == 255) & (green == 255)
    data[..., :-1][white_areas.T] = (255, 0, 0) 

    data[..., -1][~white_areas.T] = 0 

    return data

def load_cv(path):
    img = cv.imread(str(path),0)
    return img

def normalize_intensity(image):
    return image / 255.

def show(sample_df, idx):
    sample = sample_df[sample_df['slice_id'] == str.zfill(f'{idx}', 4)]

    image = load_image(sample['image'].values[0])
    label = load_image(sample['label'].values[0])

    image = normalize_intensity(image)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
    ax1.imshow(image, cmap='gray')
    ax2.imshow(label, cmap='gray')
    ax1.axis('off')
    ax2.axis('off')
    plt.subplots_adjust(wspace=0.05)
    plt.show()

def image_histogram_equalization(image, number_bins=256):
    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, density=True)
    cdf = image_histogram.cumsum() 
    cdf = (number_bins-1) * cdf / cdf[-1] 
    
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)

    return image_equalized.reshape(image.shape), cdf


def animate(sample_df, id_range):
    fig, [ax1, ax2] = plt.subplots(1, 2)
    ax1.axis('off')
    ax2.axis('off')
    images = []

    for i in tqdm(id_range):
        sample = sample_df[sample_df['slice_id'] == str.zfill(f'{i}', 4)]

        #image = load_image(sample['image'].values[0]) #v1, v2
        image = load_cv(sample['image'].values[0])
        label = load_image(sample['label'].values[0])

        #image= normalize_intensity(image) #v1
        #image= image_histogram_equalization(image)[0] #v2
        ret,image = cv.threshold(image,150,240,cv.THRESH_TRUNC) #v3

        im1 = ax1.imshow(image, animated=True, cmap='gray')
        im2 = ax2.imshow(label, animated=True, cmap='gray')
        
        if i == id_range[0]:
            ax1.imshow(image, cmap='gray')
            ax2.imshow(label, cmap='gray')
        
        images.append([im1, im2])

    ani = animation.ArtistAnimation(fig, images, interval=50, blit=True, repeat_delay=1000)
    plt.close()
    return ani



def prepare_df(im_dir='kidney_1_dense',lb_dir='kidney_1_dense'):
    df = pd.read_csv('train_rles.csv')
    base_dir = Path('train')
    subset_df = df[df.id.str.startswith(lb_dir)].reset_index(drop=True)
    subset_df['slice_id'] = subset_df['id'].map(lambda x:x.split('_')[-1]) 
    subset_df['image'] = subset_df['slice_id'].map(lambda x: base_dir / im_dir / 'images' / f'{x}.tif')
    subset_df['label'] = subset_df['slice_id'].map(lambda x: base_dir / lb_dir / 'labels' / f'{x}.tif')
    return subset_df

In [ ]:
dense_1_df = prepare_df(im_dir='kidney_1_dense',lb_dir='kidney_1_dense')

# Before

In [ ]:
ani1 = animate(dense_1_df,id_range=range(1200,1250))
ani1

# After

In [ ]:
ani2 = animate(dense_1_df,id_range=range(1200,1250))
ani2 

In [ ]:
ani3 = animate(dense_1_df,id_range=range(1200,1250))
ani3 

In [ ]:
def animate_comparison(sample_df, id_range):
    fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(18,6))
    ax1.axis('off')
    ax2.axis('off')
    ax3.axis('off')

    ax1.set_title("before")
    ax2.set_title("after")
    ax3.set_title("mask")

    images = []

    for i in tqdm(id_range):
        sample = sample_df[sample_df['slice_id'] == str.zfill(f'{i}', 4)]

        image = load_image(sample['image'].values[0])
        label = load_image(sample['label'].values[0])

        image2 = normalize_intensity(image)
        image3 = image_histogram_equalization(image)[0]  # Ensure this function returns an image

        im1 = ax1.imshow(image2, animated=True, cmap='gray')
        im2 = ax2.imshow(image3, animated=True, cmap='gray')
        im3 = ax3.imshow(label, animated=True, cmap='gray')
        
        images.append([im1, im2, im3])

    ani = animation.ArtistAnimation(fig, images, interval=50, blit=True, repeat_delay=1000)
    plt.close(fig)  # Close the plot here to prevent it from showing immediately
    return ani

animate_comparison(dense_1_df,id_range=range(1200,1300))

In [ ]:
def animate_comparison3(sample_df, id_range, figsize=(12, 6)): 
    fig, [ax1, ax2] = plt.subplots(1, 2, figsize=figsize)  
    ax1.axis('off')
    ax2.axis('off')

    ax1.set_title("before")
    ax2.set_title("after")

    images = []

    for i in tqdm(id_range):
        sample = sample_df[sample_df['slice_id'] == str.zfill(f'{i}', 4)]

        image = load_image(sample['image'].values[0])
        im = Image.open(sample['label'].values[0])
        im = im.convert('RGBA')

        data = np.array(im)  
        red, green, blue, alpha = data.T 

        white_areas = (red == 255) & (blue == 255) & (green == 255)
        data[..., :-1][white_areas.T] = (255, 0, 0) 

        data[..., -1][~white_areas.T] = 0 

        image2 = normalize_intensity(image)
        image3 = image_histogram_equalization(image)[0] 


        im1 = ax1.imshow(image2, animated=True, cmap='gray')
        im1_overlay = ax1.imshow(data, animated=True, cmap='Reds', alpha=1)

        im2 = ax2.imshow(image3, animated=True, cmap='gray')
        im2_overlay = ax2.imshow(data, animated=True, cmap='Reds', alpha=1)
        
        images.append([im1, im1_overlay, im2, im2_overlay])

    ani = animation.ArtistAnimation(fig, images, interval=50, blit=True, repeat_delay=1000)
    plt.close(fig) 
    return ani

ani = animate_comparison3(dense_1_df,id_range=range(100,2000,5))
ani

In [ ]:
matplotlib.rcParams['animation.ffmpeg_path'] = r'C:\\Users\\Bartek\\Desktop\\mgr\\ffmpeg-master-latest-win64-gpl\\bin\\ffmpeg.exe'
writergif = animation.FFMpegWriter(fps=30) 
ani.save('animation.mp4', writer=writergif, )

In [ ]:
def animate_comparison3(sample_df, id_range, figsize=(12, 6)): 
    fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=figsize)  
    ax1.axis('off')
    ax2.axis('off')

    ax1.set_title("Original")
    ax2.set_title("Mask")
    ax3.set_title("Contour")

    images = []

    for i in tqdm(id_range):
        sample = sample_df[sample_df['slice_id'] == str.zfill(f'{i}', 4)]

        img = load_cv(sample['image'].values[0])
        blur = cv.GaussianBlur(img, (25, 25), 0)
        ret,thresh = cv.threshold(blur,0,255,cv.THRESH_BINARY+ cv.THRESH_OTSU)

        contour, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        mask = np.zeros_like(img)
        cv.drawContours(mask, contour, -1, (255), 1)

        im1 = ax1.imshow(img, animated=True, cmap='gray')
        im2 = ax2.imshow(thresh, animated=True, cmap='gray')
        im3 = ax3.imshow(mask, animated=True, cmap='gray')

        images.append([im1, im2, im3])

    ani = animation.ArtistAnimation(fig, images, interval=50, blit=True, repeat_delay=1000)
    plt.close(fig) 
    return ani

ani5 = animate_comparison3(dense_1_df,id_range=range(800,1600,50))
ani5

# TEST